In [1]:
import copy
import os
import pickle

from ase.visualize import view
import numpy as np
from pymatgen.io.vasp import Poscar
from pymatgen.io.ase import AseAtomsAdaptor

from pymatgen.analysis.waterstructureCreator.load_dictionaries import (
    import_slab_structures, import_bulk_structures)

from pymatgen.analysis.waterstructureCreator.construct_supercells import (
    get_independent_Supercell_Matrices)

from pymatgen.analysis.waterstructureCreator.check_polarity import (
    get_structure_polarity)

from pymatgen.analysis.waterstructureCreator.create_slabs import (
    get_export_subslabs_clean)

from pymatgen.analysis.waterstructureCreator.create_combined_slab import (
    get_export_adsorbed_water_SS)



# First we create a list of supercell scaling matrices

While pymatgen internally allows to scan through various surface terminations and supercells, it makes more sense in the present case to fix a priori a certain supercell geometry for any (hkl) surface. Here the requirement is not that we get perfect fit with water structures but we rather want to use for a given (hkl) surface the most isotropic one, as this allows a maximum distance between periodic images and is thus the most sensible approach to choose supercell geometries, that are then only at a later stage evaluated w.r.t. to fitting 2D water supercells.



In [2]:
SuperCellSizes = [6, 12]

allredm = get_independent_Supercell_Matrices(SuperCellSizes)

#Note there is a function in 
# pymatgen.analysis.substrate_analyzer
# def gen_sl_transform_matricies(area_multiple)
# that creates all superlattice transformations
# I didnt use it as I don't really get it.
# It seems there is only superlattices with positive first
# row entries possible, why
# Now I am just not sure if these are really all
# I am not seeing how this function in combination with
# def reduce_vectors(a, b) gives all possible superlattices
# to be checked maybe with random matrices
# Question is reduce_vectors(reduce_vectors(Superlattices)*Basis)
# all there is with the pymatgen functionality?

os.makedirs('pickle_data', exist_ok=True)
with open("pickle_data/all_independent_supercells_{}-{}.pickle".format(SuperCellSizes[0], SuperCellSizes[1]), "wb") as o:
    pickle.dump(allredm,o) 

10000  supercell matrices created
20000  supercell matrices created


# First we need a bunch of bulk cells in the conventional unit cell and the 2D water structures
These can be loaded from files with these functions

In [3]:
with open("pickle_data/all_independent_supercells_{}-{}.pickle".format(SuperCellSizes[0], SuperCellSizes[1]), "rb") as o:
    allredm = pickle.load(o) 
    
films = import_slab_structures(filename="h2o_export.json")

bulk_structs = import_bulk_structures(filename="bulk_export.json")

print("Materials:", bulk_structs.keys())
print("=++++++++++++++++++++++++++++++++=")
print("Water phases:", films.keys())
print("=++++++++++++++++++++++++++++++++=")

print("Each water phase then has {} different possible low energy lattice constants.".format(len(films['p3b4o1'])) )
print("These are")
for l in films['p3b4o1']:
    print("cell lengths: ", np.round(l[0][0],3))

public version
Materials: dict_keys(['Pd', 'Cu', 'Au', 'Pt', 'Ag'])
=++++++++++++++++++++++++++++++++=
Water phases: dict_keys(['p4b2o3', 'p3b1o3', 'p5b4o1', 'p2b3o3', 'p5b2o4', 'p3b4o2', 'p3b2o3', 'p3b4o1'])
=++++++++++++++++++++++++++++++++=
Each water phase then has 20 different possible low energy lattice constants.
These are
cell lengths:  [3.872 7.441]
cell lengths:  [4.259 5.082]
cell lengths:  [4.259 5.59 ]
cell lengths:  [4.259 6.149]
cell lengths:  [4.259 6.764]
cell lengths:  [4.259 7.441]
cell lengths:  [4.259 8.185]
cell lengths:  [4.259 9.003]
cell lengths:  [4.685 5.082]
cell lengths:  [4.685 5.59 ]
cell lengths:  [4.685 6.149]
cell lengths:  [4.685 6.764]
cell lengths:  [4.685 7.441]
cell lengths:  [4.685 8.185]
cell lengths:  [4.685 9.003]
cell lengths:  [5.154 5.59 ]
cell lengths:  [5.154 6.149]
cell lengths:  [5.154 6.764]
cell lengths:  [5.154 7.441]
cell lengths:  [5.154 8.185]


In [4]:
"""

import os 
old_names = ['rh2_lower', 'f_sq', 'double_hex', 'rh2_upper', 'double_hex2', 'sq4_alt1', 'sq4_alt2', 'sq4']
new_names = ['p4b2o3', 'p3b1o3', 'p5b4o1', 'p2b3o3', 'p5b2o4', 'p3b4o2', 'p3b2o3', 'p3b4o1']

old_names = [i+'_xy' for i in old_names]
new_names = [i+'_xy' for i in new_names]


for filename in os.listdir('water_slabs/'):
    filename_new=None
    for i in range(len(old_names)):
        if old_names[i] in filename:
            filename_new = filename.replace(old_names[i], new_names[i])
    if filename_new==None:
        print('Error')
    print(filename_new)
    os.rename('water_slabs/{}'.format(filename), 'water_slabs/{}'.format(filename_new))

"""

"\n\nimport os \nold_names = ['rh2_lower', 'f_sq', 'double_hex', 'rh2_upper', 'double_hex2', 'sq4_alt1', 'sq4_alt2', 'sq4']\nnew_names = ['p4b2o3', 'p3b1o3', 'p5b4o1', 'p2b3o3', 'p5b2o4', 'p3b4o2', 'p3b2o3', 'p3b4o1']\n\nold_names = [i+'_xy' for i in old_names]\nnew_names = [i+'_xy' for i in new_names]\n\n\nfor filename in os.listdir('water_slabs/'):\n    filename_new=None\n    for i in range(len(old_names)):\n        if old_names[i] in filename:\n            filename_new = filename.replace(old_names[i], new_names[i])\n    if filename_new==None:\n        print('Error')\n    print(filename_new)\n    os.rename('water_slabs/{}'.format(filename), 'water_slabs/{}'.format(filename_new))\n\n"

In [5]:
"""

old_names = ['rh2_lower', 'f_sq', 'double_hex', 'rh2_upper', 'double_hex2', 'sq4_alt1', 'sq4_alt2', 'sq4']
new_names = ['p4b2o3', 'p3b1o3', 'p5b4o1', 'p2b3o3', 'p5b2o4', 'p3b4o2', 'p3b2o3', 'p3b4o1']

old_names = [i+'_xy' for i in old_names]
new_names = [i+'_xy' for i in new_names]

import json
with open('./h2o_export.json', "r") as o:
    data = json.load(o)
for keys in data.keys():
    for i in range(len(data[keys])):
        filename = data[keys][i][-1]
        filename_new=None
        for j in range(len(old_names)):
            if old_names[j] in filename:
                filename_new = filename.replace(old_names[j], new_names[j])
        if filename_new==None:
            print(filename_new)
            print(data[keys][i])
            print('Error')
        data[keys][i][-1] = filename_new

with open('h2o_export.json', 'w') as fp:
    json.dump(data, fp)
"""

'\n\nold_names = [\'rh2_lower\', \'f_sq\', \'double_hex\', \'rh2_upper\', \'double_hex2\', \'sq4_alt1\', \'sq4_alt2\', \'sq4\']\nnew_names = [\'p4b2o3\', \'p3b1o3\', \'p5b4o1\', \'p2b3o3\', \'p5b2o4\', \'p3b4o2\', \'p3b2o3\', \'p3b4o1\']\n\nold_names = [i+\'_xy\' for i in old_names]\nnew_names = [i+\'_xy\' for i in new_names]\n\nimport json\nwith open(\'./h2o_export.json\', "r") as o:\n    data = json.load(o)\nfor keys in data.keys():\n    for i in range(len(data[keys])):\n        filename = data[keys][i][-1]\n        filename_new=None\n        for j in range(len(old_names)):\n            if old_names[j] in filename:\n                filename_new = filename.replace(old_names[j], new_names[j])\n        if filename_new==None:\n            print(filename_new)\n            print(data[keys][i])\n            print(\'Error\')\n        data[keys][i][-1] = filename_new\n\nwith open(\'h2o_export.json\', \'w\') as fp:\n    json.dump(data, fp)\n'

# Create interface Structures

In [ ]:
# This is where we will place the structures
folder = "created_interface_structures_supercells_{}-{}/".format(SuperCellSizes[0], SuperCellSizes[1])
if not os.path.exists(os.getcwd() + "/" + folder):
    os.mkdir(os.getcwd() + "/" + folder)

# Are water 2D phases polar or not. IF polar both orientations will be placed onto the surface later
films_polarity = get_structure_polarity(films)  

# Supercell sizes we are interested in
SuperCellSizes = [6, 12] 
target_ss =  list(np.arange(SuperCellSizes[0], \
                            SuperCellSizes[1] + 1))   


limitxylen = 36. # limit on any cell length in Angstrom, in order to ignore directly all very assymmetric cells
horizontaldist = 2. # the distance with which we scan the different lateral arrangements in the unitcell
waterdistance = 2.3 # The distance to the surface where we want to place the waters


# all results will be placed in this dictionary
all_singlesided_structures = {}
          

# Here we choose which surfaces and how thick we want the cells to be.
# pymatgen has problems with thickness in dhkl units. This works fine
# This will create 100 slabs with 6 atomic layers, 110 with 8 layers and 111 with 6 layers
# later in the process we will cut away the lower half of the slab for prerelaxations and
# this gives us 3,4,3 layers in asymmetric cells sizes 
interface_specs = [ [[1,0,0], "(100)", 3], \
                   [[1,1,0] ,"(110)", 4],  \
                   [[1,1,1], "(111)", 6] ]
#names
allNs= {i[1]: target_ss for i in interface_specs} 
print(allNs)
#raise


for material in ['Pt']: # bulk_structs.keys(), Careful you need always conventional cells! for hkl to be correct
    bulk_struct = bulk_structs[material]
    print(material)
    all_singlesided_structures[material] = {}
    if True:
        all_primitive_slabs = [] 
        # First from conventional bulk structure construct the slabs (primitive slabs)
        # include the oriented primitive bulk structures
        # calculation of bulk references in similar geom makes convergence good (interface energy)
        
        for miller, millername, lays in interface_specs:
            if True:
                all_singlesided_structures[material][millername] = {}
                all_singlesided_structures[material][millername][lays] = {}

                if True:  
                    cleanslab_dict, primitive_slab = \
                    get_export_subslabs_clean(bulk_struct,\
                                          miller, millername,lays, material, folder, vacuum = 17)
                    print("primitive", millername, primitive_slab.lattice.matrix)
                    #view(AseAtomsAdaptor.get_atoms(structure=primitive_slab))
                    #raise SystemExit('sfsbn')
                    
                    all_singlesided_structures[material][millername][lays]["clean"] = cleanslab_dict

                    for target_N in allNs[millername]:
                        print('target_N', target_N)
                   
                        for i in range(2):
                            print('######################')
                        print("Analysing surface {} for {} target surface atoms".format(miller,target_N))
                        for i in range(2):
                            print('######################')
                        returned_SS_infos = get_export_adsorbed_water_SS(target_N, primitive_slab, \
                                                 allredm, films, all_singlesided_structures, \
                                                 material, folder, millername, lays,\
                                                     horizontaldist, waterdistance,\
                                                     limitxylen, \
                                                     films_polarity)
                        #raise Systemerror('Stop')
                        all_singlesided_structures[material][millername][lays][target_N]["all_supercells_info"] = returned_SS_infos

                        
with open("pickle_data/all_singlesided_structures_target_cells_larger_{}-{}.pickle".format(SuperCellSizes[0], SuperCellSizes[1]), "wb") as o:
    pickle.dump(all_singlesided_structures, o)
                
                
            
             


{'(100)': [6, 7, 8, 9, 10, 11, 12], '(110)': [6, 7, 8, 9, 10, 11, 12], '(111)': [6, 7, 8, 9, 10, 11, 12]}
Pt
THIS VALUE SHOULD BE 1 ALWAYS 1.0000000000000002
primitive (100) [[2.80274925e+00 0.00000000e+00 1.71618895e-16]
 [4.50716455e-16 2.80274925e+00 1.71618895e-16]
 [0.00000000e+00 0.00000000e+00 2.69092150e+01]]
target_N 6
######################
######################
Analysing surface [1, 0, 0] for 6 target surface atoms
######################
######################


/media/thorben/Data/Butterfly/DFTprojects/Git/WaterStructureCreator/code/pymatgen-2020.11.11/pymatgen/analysis/waterstructureCreator/create_slabs.py:122: FutureWarning: from_lengths_and_angles is deprecated
Use Lattice.from_parameters instead. This will be removed in v2020.*
  Lattice.from_lengths_and_angles(
/home/thorben/Butterfly/DFTroot/Python/miniconda3/envs/python_v1/lib/python3.8/site-packages/matplotlib/text.py:1150: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if s != self._text:


target_N 7
######################
######################
Analysing surface [1, 0, 0] for 7 target surface atoms
######################
######################


/home/thorben/Butterfly/DFTroot/Python/miniconda3/envs/python_v1/lib/python3.8/site-packages/matplotlib/text.py:1150: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if s != self._text:


target_N 8
######################
######################
Analysing surface [1, 0, 0] for 8 target surface atoms
######################
######################


...

create_all_interfaces
[Structure Summary
Lattice
    abc : 9.709008200613285 11.891058000000001 30.928262371829646
 angles : 89.99999999999999 90.0 74.20683095173604
 volume : 3435.888078172105
      A : -5.605498498188363 7.927372 1.4217374760093383e-16
      B : -11.210996996376728 -3.963686000000001 -9.291813479794265e-16
      C : -7.098006601745049e-15 -7.098006601745049e-15 30.928262371829646
PeriodicSite: Pt (-12.6124, 1.9818, 10.5593) [0.6500, 0.8000, 0.3414]
PeriodicSite: Pt (-7.0069, -1.9818, 10.5593) [0.0500, 0.6000, 0.3414]
PeriodicSite: Pt (-7.0069, 1.9818, 10.5593) [0.4500, 0.4000, 0.3414]
PeriodicSite: Pt (-9.8096, -1.9818, 10.5593) [0.1500, 0.8000, 0.3414]
PeriodicSite: Pt (-9.8096, 1.9818, 10.5593) [0.5500, 0.6000, 0.3414]
PeriodicSite: Pt (-9.8096, 5.9455, 10.5593) [0.9500, 0.4000, 0.3414]
PeriodicSite: Pt (-4.2041, 1.9818, 10.5593) [0.3500, 0.2000, 0.3414]
PeriodicSite: Pt (-4.2041, 5.9455, 10.5593) [0.7500, 0.0000, 0.3414]
PeriodicSite: Pt (-7.0069, 5.9455, 10.5593)

/home/thorben/Butterfly/DFTroot/Python/miniconda3/envs/python_v1/lib/python3.8/site-packages/matplotlib/text.py:1150: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if s != self._text:
/media/thorben/Data/Butterfly/DFTprojects/Git/WaterStructureCreator/code/pymatgen-2020.11.11/pymatgen/analysis/waterstructureCreator/interface_mod.py:451: UserWarning: Check generated structure, it may contain atoms too closely placed
  warnings.warn("Check generated structure, it may contain atoms too closely placed")


create_all_interfaces
[Structure Summary
Lattice
    abc : 11.556031196135894 11.891058000000001 30.94542237182964
 angles : 89.99999999999999 90.0 62.78507729277374
 volume : 3781.573861971471
      A : -8.408247747282545 7.927372 -2.9445147234858033e-17
      B : -11.210996996376728 -3.963686000000001 -9.291813479794263e-16
      C : -7.098006601745049e-15 -7.098006601745048e-15 30.94542237182964
PeriodicSite: Pt (-7.0069, -1.9818, 10.5679) [0.0455, 0.5909, 0.3415]
PeriodicSite: Pt (-7.0069, 1.9818, 10.5679) [0.4091, 0.3182, 0.3415]
PeriodicSite: Pt (-7.0069, 5.9455, 10.5679) [0.7727, 0.0455, 0.3415]
PeriodicSite: Pt (-12.6124, 1.9818, 10.5679) [0.5909, 0.6818, 0.3415]
PeriodicSite: Pt (-12.6124, 5.9455, 10.5679) [0.9545, 0.4091, 0.3415]
PeriodicSite: Pt (-4.2041, 1.9818, 10.5679) [0.3182, 0.1364, 0.3415]
PeriodicSite: Pt (-15.4151, 1.9818, 10.5679) [0.6818, 0.8636, 0.3415]
PeriodicSite: Pt (-9.8096, -1.9818, 10.5679) [0.1364, 0.7727, 0.3415]
PeriodicSite: Pt (-9.8096, 1.9818, 10.567

/home/thorben/Butterfly/DFTroot/Python/miniconda3/envs/python_v1/lib/python3.8/site-packages/matplotlib/text.py:1150: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if s != self._text:
/media/thorben/Data/Butterfly/DFTprojects/Git/WaterStructureCreator/code/pymatgen-2020.11.11/pymatgen/analysis/waterstructureCreator/interface_mod.py:451: UserWarning: Check generated structure, it may contain atoms too closely placed
  warnings.warn("Check generated structure, it may contain atoms too closely placed")


create_all_interfaces
[Structure Summary
Lattice
    abc : 11.210996996376728 11.891058000000001 31.000622371829664
 angles : 90.00000000000001 90.00000000000001 90.0
 volume : 4132.712049945681
      A : 11.210996996376728 0.0 6.864755793431676e-16
      B : 1.912227969841857e-15 11.891058000000001 7.281173059087764e-16
      C : -7.098006601745049e-15 -7.098006601745048e-15 31.000622371829664
PeriodicSite: Pt (1.4014, 1.9818, 10.5955) [0.1250, 0.1667, 0.3418]
PeriodicSite: Pt (1.4014, 5.9455, 10.5955) [0.1250, 0.5000, 0.3418]
PeriodicSite: Pt (1.4014, 9.9092, 10.5955) [0.1250, 0.8333, 0.3418]
PeriodicSite: Pt (4.2041, 1.9818, 10.5955) [0.3750, 0.1667, 0.3418]
PeriodicSite: Pt (4.2041, 5.9455, 10.5955) [0.3750, 0.5000, 0.3418]
PeriodicSite: Pt (4.2041, 9.9092, 10.5955) [0.3750, 0.8333, 0.3418]
PeriodicSite: Pt (7.0069, 1.9818, 10.5955) [0.6250, 0.1667, 0.3418]
PeriodicSite: Pt (7.0069, 5.9455, 10.5955) [0.6250, 0.5000, 0.3418]
PeriodicSite: Pt (7.0069, 9.9092, 10.5955) [0.6250, 0.8333

In [ ]:
raise Systemerror('Stop')

In [ ]:
from ase.io import read
traj=[]
for file in os.listdir('./created_interface_structures_supercells_6-12/'):
    if file.startswith('Pt_combined_(111)_12'):
        atoms = read('./created_interface_structures_supercells_6-12/{}'.format(file))
        ind_O = np.array([atom.index for atom in atoms if (atom.symbol == 'O')])
        
        if len(ind_O)!=8:
            continue
        traj+=[atoms]
print(len(traj))

In [ ]:
returned_SS_infos

In [ ]:
print(all_singlesided_structures.keys())
#Logic of the keys:
# material, surface, thickness, SupercellSize=number_of_surface_atoms, water_structure_name, 
print(all_singlesided_structures['Pt']['(111)'][6][12]['p3b4o1'].keys())

# So there are 3 different water lattice vectors for rh2_lower phase fitting this surface
# these have indices 1, 13, 15

#print(all_singlesided_structures['Pt']['(111)'][6][12]['p3b4o1'][1]['combined_half'].keys())

# water structure 1 is then put with 4 different lateral shifts onto the surface
# It can be found in folder under the name

filename = all_singlesided_structures['Pt']['(111)'][6][12]['p3b4o1'][1]['combined_half'][1]['combined_half']
print(filename)

#Let's look at it:



view(AseAtomsAdaptor.get_atoms(Poscar.from_file(filename).structure))


#Let's chek the others:

filename = all_singlesided_structures['Pt']['(111)'][6][12]['p3b4o1'][13]['combined_half'][1]['combined_half']
print(filename)
view(AseAtomsAdaptor.get_atoms(Poscar.from_file(filename).structure))


filename = all_singlesided_structures['Pt']['(111)'][6][12]['p3b4o1'][14]['combined_half'][1]['combined_half']
print(filename)
view(AseAtomsAdaptor.get_atoms(Poscar.from_file(filename).structure))

# there is a small problem that 13/14 are equivalent (this is due to the matching algorithm)
# But 1 and 13 are clearly different (check e.g. nr of waters / water density)
# Currently there is no standardization where atoms sit


#Pt_sub_(111)_slab_half.vasp


In [ ]:


selection = [ ['(111)',6, 12] , ['(100)',3, 9], ['(110)',4, 6]]



for s in selection:
    k = copy.deepcopy(all_singlesided_structures['Pt'][s[0]][s[1]][s[2]]['p5b4o1'])
    k.pop('is_polar') 
    for kl in k.keys():
        print(s, kl, all_singlesided_structures['Pt'][s[0]][s[1]][s[2]]['p5b4o1'][kl]['number_of_water_molecules'] )
        if kl in [13,14]:
            ss = all_singlesided_structures['Pt'][s[0]][s[1]][s[2]]['p5b4o1'][kl]['combined_half'][1]['combined_half']
            view(AseAtomsAdaptor.get_atoms(Poscar.from_file(ss).structure))


